# Library

In [163]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
from datetime import datetime

In [55]:
# 소수점 옵션
pd.options.display.float_format = '{:.2f}'.format
# pd.reset_option('display.float_format')

# fold path

In [56]:
ods_fold = 'D:/data/big2/ODS'

# Load

## 등록&제원(말소제거)

In [160]:
grade = 4

In [6]:
file_name = f'[ODS][G{grade}]등록정보&제원정보(말소제거)(2022.12.28).csv'

In [7]:
# about 32s
car_info = pd.read_csv(os.path.join(ods_fold, file_name), encoding='cp949', low_memory=False)
car_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1511251 entries, 0 to 1511250
Data columns (total 49 columns):
 #   Column        Non-Null Count    Dtype  
---  ------        --------------    -----  
 0   차량관리번호        1511251 non-null  object 
 1   차량등록번호        1511251 non-null  object 
 2   본거지법정동코드      1511251 non-null  int64  
 3   소유자구분         1511251 non-null  object 
 4   차종_등록정보       1511249 non-null  object 
 5   용도            1511251 non-null  object 
 6   현재회원번호        1510412 non-null  object 
 7   차대번호          1511251 non-null  object 
 8   최초등록일자        1511251 non-null  int64  
 9   차량연식          1511251 non-null  int64  
 10  차량제작일자        1342461 non-null  float64
 11  배출가스검사유효일자    1511239 non-null  float64
 12  제원관리번호        1511251 non-null  object 
 13  배출가스인증번호      1387530 non-null  object 
 14  배출가스등급        1511251 non-null  object 
 15  저공해조치YN(미사용)  1511096 non-null  object 
 16  저공해조치종류       189 non-null      float64
 17  저공해조치구조변경YN   1511251 non-n

## 저감장치 부착이력(중복차대제거)

In [10]:
file_name = '[ODS]저감장치부착이력(중복차대제거)(2022.12.28).csv'

In [12]:
dpf = pd.read_csv(os.path.join(ods_fold, file_name), low_memory=False, encoding='cp949')
dpf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 809255 entries, 0 to 809254
Data columns (total 11 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   저감장치종류      585410 non-null  object 
 1   TDMS부착여부    204903 non-null  object 
 2   구조변경일자      809255 non-null  int64  
 3   부착업체명       800321 non-null  object 
 4   부착일자        809254 non-null  float64
 5   탈거승인일       169 non-null     float64
 6   차량번호        809255 non-null  object 
 7   차대번호        809255 non-null  object 
 8   저감장치반납여부    324650 non-null  object 
 9   저감장치구분      809255 non-null  object 
 10  삼원촉매장치반납여부  13688 non-null   object 
dtypes: float64(2), int64(1), object(8)
memory usage: 67.9+ MB


## 유로4인증대장

In [13]:
# 유로4인증대장
euro = pd.read_excel('D:/data/big2/dpf.xlsx')
euro.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82 entries, 0 to 81
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   배출가스인증번호  82 non-null     object
 1   DPF유무     82 non-null     object
dtypes: object(2)
memory usage: 1.4+ KB


## 본거지법정동코드

In [203]:
code = pd.read_excel('D:/data/big2/법정동코드_2022.10.14.xlsx')
code.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46340 entries, 0 to 46339
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   법정동코드     46340 non-null  int64  
 1   시도        46340 non-null  object 
 2   시구군       46317 non-null  object 
 3   읍면동       45847 non-null  object 
 4   리         33682 non-null  object 
 5   지역3       0 non-null      float64
 6   폐지여부      46340 non-null  object 
 7   대기관리권역    46340 non-null  object 
 8   대기관리권역YN  46334 non-null  object 
dtypes: float64(1), int64(1), object(7)
memory usage: 3.2+ MB


# 전처리
- 말소제거
- 중복 차대 제거
- 기타연료 변경
    - 다른 연료 샘플과 '자동차형식', '제원관리번호'로 매칭
- 저감장치부착이력(차대번호 기준), 유로4인증대장(배인번호 기준)과 병합
- 저감장치구분 중 '1종, 1종+SCR'만 DPF 장착으로 입력
- 본거지법정동코드(법정동코드 기준)와 병합

## 말소제거

In [57]:
car_info['차량말소YN'].value_counts(dropna=False)

N    1511251
Name: 차량말소YN, dtype: int64

## 중복 차대번호 제거
- 최종등록일자, 배출가스검사유효일자 최신 데이터 사용

In [58]:
car_info1 = car_info.sort_values(['최초등록일자', '배출가스검사유효일자'], ascending=[False, False]).drop_duplicates('차대번호').reset_index(drop=True)
car_info1.shape

(1511211, 49)

## 기타연료 처리
- '제원관리번호'로 매칭
- '자동차형식' 으로 매칭
- '엔진형식' 으로 매칭

In [59]:
car_info1['연료'].value_counts(dropna=False)

경유             1152697
휘발유             332313
LPG(액화석유가스)      26089
NaN                 71
기타연료                39
CNG(압축천연가스)          1
알코올                  1
Name: 연료, dtype: int64

In [108]:
etc_car = car_info1[(car_info1['연료'] == '기타연료') | (car_info1['연료'].isnull() == True)]
etc_car.shape

(110, 49)

In [109]:
etc_car['연료'].value_counts(dropna=False)

NaN     71
기타연료    39
Name: 연료, dtype: int64

In [110]:
remain_car = car_info1[(car_info1['연료'] != '기타연료') & (car_info1['연료'].isnull() != True)]
remain_car.shape

(1511101, 49)

In [111]:
remain_car['연료'].value_counts(dropna=False)

경유             1152697
휘발유             332313
LPG(액화석유가스)      26089
CNG(압축천연가스)          1
알코올                  1
Name: 연료, dtype: int64

In [112]:
etc_car['제원관리번호'].isnull().sum()

0

### 제원관리번호로 매칭

In [113]:
len(set(etc_car['제원관리번호'].unique()) & set(remain_car['제원관리번호'].unique()))

0

### 자동차형식으로 매칭
- 이슈 : 'B250' 휘발유, LPG(액화석유가스) 두가지가 존재
    - 휘발유 76개, LPG(액화석유가스) 1개 -> '휘발유'로 변경

In [114]:
remain_car.loc[remain_car['자동차형식'] == 'B250', '연료'].unique()

array(['휘발유', 'LPG(액화석유가스)'], dtype=object)

In [115]:
remain_car.loc[remain_car['자동차형식'] == 'B250', '연료'].value_counts(dropna=False)

휘발유            76
LPG(액화석유가스)     1
Name: 연료, dtype: int64

In [116]:
remain_car.loc[remain_car['자동차형식'] == 'B250', '연료'] = '휘발유'

In [117]:
remain_car.loc[remain_car['자동차형식'] == 'B250', '연료'].value_counts(dropna=False)

휘발유    77
Name: 연료, dtype: int64

In [118]:
etc_car['자동차형식'].isnull().sum()

40

In [119]:
etc_car[['자동차형식', '연료']].head()

,자동차형식,연료
1028,JZA 80,NaN
3398,L4DC,기타연료
457123,NaN,NaN
535174,NaN,NaN
538634,NaN,NaN


In [120]:
remain_car.loc[remain_car['자동차형식'] == 'JZA 80', '연료']

Series([], Name: 연료, dtype: object)

In [121]:
remain_car.loc[remain_car['자동차형식'] == 'B250', '연료'].shape

(77,)

In [122]:
uni_carform_etc = list(set(etc_car['자동차형식'].unique()) & set(remain_car['자동차형식'].unique()))
len(uni_carform_etc)

28

In [123]:
uni_carform_etc

[nan,
 '911터보',
 'GR',
 'FD3S',
 'MS122',
 '740GLE',
 'MR2',
 'MSV3-27A',
 'B250',
 'ST-34',
 'SR19F',
 'TF68E',
 '140 051',
 '1994',
 'FE-26HT',
 'JA19Y',
 '202 028',
 '154AN55A',
 'TF19I',
 'WA19B',
 'FXB',
 'TF19T',
 'FXA-1',
 'B5-25SX',
 '210 037',
 'FXA',
 'VG30',
 '964']

In [124]:
uni_carform_etc = uni_carform_etc[1:]
uni_carform_etc

['911터보',
 'GR',
 'FD3S',
 'MS122',
 '740GLE',
 'MR2',
 'MSV3-27A',
 'B250',
 'ST-34',
 'SR19F',
 'TF68E',
 '140 051',
 '1994',
 'FE-26HT',
 'JA19Y',
 '202 028',
 '154AN55A',
 'TF19I',
 'WA19B',
 'FXB',
 'TF19T',
 'FXA-1',
 'B5-25SX',
 '210 037',
 'FXA',
 'VG30',
 '964']

In [88]:
dict_fuel_by_carform = {}
for one in uni_carform_etc:
    dict_fuel_by_carform[one] = remain_car[remain_car['자동차형식'] == one]['연료'].unique()[0]
dict_fuel_by_carform

{'911터보': '휘발유',
 'GR': '휘발유',
 'FD3S': '휘발유',
 'MS122': '휘발유',
 '740GLE': '휘발유',
 'MR2': '휘발유',
 'MSV3-27A': '휘발유',
 'B250': '휘발유',
 'ST-34': '휘발유',
 'SR19F': '휘발유',
 'TF68E': '휘발유',
 '140 051': '휘발유',
 '1994': '휘발유',
 'FE-26HT': '휘발유',
 'JA19Y': '휘발유',
 '202 028': '휘발유',
 '154AN55A': '휘발유',
 'TF19I': '휘발유',
 'WA19B': '휘발유',
 'FXB': '휘발유',
 'TF19T': '휘발유',
 'FXA-1': '휘발유',
 'B5-25SX': '휘발유',
 '210 037': '휘발유',
 'FXA': '휘발유',
 'VG30': '휘발유',
 '964': '휘발유'}

### 엔진형식으로 매칭

In [125]:
uni_engineform_etc = list(set(etc_car['엔진형식'].unique()) & set(remain_car['엔진형식'].unique()))
len(uni_engineform_etc)

28

In [126]:
uni_engineform_etc

[nan,
 'C32A1',
 'B5',
 'F8C',
 'M64/02',
 'MSV3',
 'B230F',
 'VQ30',
 'G15CF',
 'V6',
 '119',
 'G15MF',
 '112 944',
 '7M',
 'G16SF',
 'FE',
 '102 962',
 '834C.146',
 '5E',
 '111 970',
 'G4AJ',
 'B',
 'SR20',
 '2JZ',
 'H20MR',
 '104 941',
 'G4CP',
 'M30-69']

In [127]:
uni_engineform_etc = uni_engineform_etc[1:]
uni_engineform_etc

['C32A1',
 'B5',
 'F8C',
 'M64/02',
 'MSV3',
 'B230F',
 'VQ30',
 'G15CF',
 'V6',
 '119',
 'G15MF',
 '112 944',
 '7M',
 'G16SF',
 'FE',
 '102 962',
 '834C.146',
 '5E',
 '111 970',
 'G4AJ',
 'B',
 'SR20',
 '2JZ',
 'H20MR',
 '104 941',
 'G4CP',
 'M30-69']

In [128]:
dict_fuel_by_engineform = {}
for one in uni_engineform_etc:
    dict_fuel_by_engineform[one] = remain_car[remain_car['엔진형식'] == one]['연료'].unique()[0]
dict_fuel_by_engineform

{'C32A1': '휘발유',
 'B5': '휘발유',
 'F8C': '휘발유',
 'M64/02': '휘발유',
 'MSV3': '휘발유',
 'B230F': '휘발유',
 'VQ30': '휘발유',
 'G15CF': '휘발유',
 'V6': '휘발유',
 '119': '휘발유',
 'G15MF': '휘발유',
 '112 944': '휘발유',
 '7M': '휘발유',
 'G16SF': '휘발유',
 'FE': '휘발유',
 '102 962': '휘발유',
 '834C.146': '휘발유',
 '5E': '휘발유',
 '111 970': '휘발유',
 'G4AJ': '휘발유',
 'B': '휘발유',
 'SR20': '휘발유',
 '2JZ': '휘발유',
 'H20MR': '휘발유',
 '104 941': '휘발유',
 'G4CP': '휘발유',
 'M30-69': '휘발유'}

In [133]:
fuel_list = []
for one, two, three in etc_car[['자동차형식', '엔진형식', '연료']].values:
    if one in dict_fuel_by_carform.keys():
        fuel_list.append(dict_fuel_by_carform[one])
    else:
        if two in dict_fuel_by_engineform.keys():
            fuel_list.append(dict_fuel_by_engineform[two])
        else:
            fuel_list.append(three)
len(fuel_list)

110

In [134]:
etc_car['연료'] = fuel_list
etc_car['연료'].value_counts(dropna=False)

C:\Users\kbjung\AppData\Local\Temp\ipykernel_14584\1695469869.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  etc_car['연료'] = fuel_list


휘발유     67
NaN     40
기타연료     3
Name: 연료, dtype: int64

In [135]:
etc_car.loc[etc_car['자동차형식'] == 'B250', '연료']

1228842    휘발유
Name: 연료, dtype: object

In [136]:
etc_car.loc[etc_car['엔진형식'] == 'C32A1', '연료']

1167945    휘발유
Name: 연료, dtype: object

In [137]:
# 병합
car_info2 = pd.concat([etc_car, remain_car], ignore_index=True)
car_info2.shape

(1511211, 49)

## 저감장치부착이력 병합
- 차대번호 기준

In [143]:
car_dpf = car_info2.merge(dpf, on='차대번호', how='left')
car_dpf.shape

(1511211, 59)

## 유로4인증대장 병합
- 배인번호 기준

In [205]:
car_euro = car_dpf.merge(euro, on='배출가스인증번호', how='left')
car_euro.shape

(1511211, 60)

In [206]:
car_euro.columns

Index(['차량관리번호', '차량등록번호', '본거지법정동코드', '소유자구분', '차종_등록정보', '용도', '현재회원번호',
       '차대번호', '최초등록일자', '차량연식', '차량제작일자', '배출가스검사유효일자', '제원관리번호', '배출가스인증번호',
       '배출가스등급', '저공해조치YN(미사용)', '저공해조치종류', '저공해조치구조변경YN', '차량등록번호숫자정보',
       '차량말소YN', '최초회원번호', '취득일자', '취득금액', '생성일시_등록정보', '수정일시_등록정보', '차명',
       '차종_제원정보', '차종분류', '차종유형(소분류)', '자동차형식', '제작사명', '연료', '엔진형식', '공차중량',
       '총중량', '적재중량', '엔진출력', '배기량', '승차인원', '차량길이', '차량너비', '차량높이', '원산지명',
       '삭제YN', '승인일자', '구동형식', '변속기종류', '생성일시_제원정보', '수정일시_제원정보', '저감장치종류',
       'TDMS부착여부', '구조변경일자', '부착업체명', '부착일자', '탈거승인일', '차량번호', '저감장치반납여부',
       '저감장치구분', '삼원촉매장치반납여부', 'DPF유무'],
      dtype='object')

In [207]:
car_euro['DPF유무'].value_counts(dropna=False)

무      686158
NaN    518957
유      306096
Name: DPF유무, dtype: int64

In [208]:
car_euro['DPF유무'] = car_euro['DPF유무'].fillna('무')
car_euro['DPF유무'].value_counts(dropna=False)

무    1205115
유     306096
Name: DPF유무, dtype: int64

## 본거지법정동 코드 병합
- 본거지법정동코드, 법정동코드 기준

In [209]:
df = car_euro.merge(code, left_on='본거지법정동코드', right_on='법정동코드', how='left')
df.shape

(1511211, 69)

In [210]:
df['본거지법정동코드'].isnull().sum()

0

In [234]:
df['본거지법정동코드'] = df['본거지법정동코드'].astype('str')

In [235]:
df['행정구역코드'] = df['본거지법정동코드'].str[:5]
df['행정구역코드'].head()

0    41220
1    48740
2    11170
3    47850
4    47850
Name: 행정구역코드, dtype: object

# 분석

In [213]:
df['연료'].unique()

array([nan, '기타연료', '휘발유', '경유', 'LPG(액화석유가스)', 'CNG(압축천연가스)', '알코올'],
      dtype=object)

In [216]:
df[ (df['연료'] == '휘발유_가스') | (df['연료'] == '알코올') | (df['연료'] == 'LPG(액화석유가스)') | (df['연료'] == 'CNG(압축천연가스)') ].shape

(26090, 69)

In [217]:
df[df['연료'] == '경유'].shape

(1152697, 69)

In [218]:
fuel_list = []
for one in tqdm(df['연료']):
    if one == '휘발유' or one == '알코올' or one == 'LPG(액화석유가스)' or one == 'CNG(압축천연가스)':
        fuel_list.append('휘발유_가스')
    elif one == '경유':
        fuel_list.append('경유')
    else:
        fuel_list.append('기타')
len(fuel_list)

100%|██████████| 1511211/1511211 [00:00<00:00, 1697831.86it/s]


1511211

In [219]:
df['fuel'] = fuel_list

In [220]:
df['fuel'].value_counts()

경유        1152697
휘발유_가스     358471
기타             43
Name: fuel, dtype: int64

In [221]:
grade_list = []
for f, y, cy, d, e in tqdm(zip(df['fuel'], df['차량제작일자'], df['차량연식'],df['저감장치구분'], df['DPF유무'])):
    if (f == '휘발유_가스') and ( (19980101 <= y <= 20001231) or (1998 <= cy <= 2000) ):
        grade_list.append('A')
    elif (f == '휘발유_가스') and ( (y <= 19971231) or (cy <= 1997) ):
        grade_list.append('B')
    elif (f == '경유') and ( (y >= 20080101) or (cy >= 2008) ) and ( (str(d) == '1종') or (str(d) == '1종+SCR') or (e == '유') ):
        grade_list.append('A')
    elif (f == '경유') and ( (y <= 20071231) or (cy <= 2007) )and ( (str(d) == '1종') or (str(d) == '1종+SCR') or (e == '유') ):
        grade_list.append('B')
    elif (f == '경유') and ( (y >= 20080101) or (cy >= 2008) ) and ( (str(d) == 'nan') or (str(d) == '2종') or (e == '무') ):
        grade_list.append('C')
    elif (f == '경유') and ( (y <= 20071231) or (cy <= 2007) ) and ( (str(d) == 'nan') or (str(d) == '2종') or (e == '무') ):
        grade_list.append('D')
    else:
        grade_list.append('X')
len(grade_list)

1511211it [00:02, 505623.00it/s]


1511211

In [222]:
df['Grade'] = grade_list

In [223]:
num_by_fuel = df.groupby(['fuel'])['차대번호'].count().reset_index().sort_values('차대번호', ascending=False)
num_by_fuel = num_by_fuel.rename(columns={'차대번호':'대수'})
num_by_fuel

,fuel,대수
0,경유,1152697
2,휘발유_가스,358471
1,기타,43


In [224]:
num_by_grade = df.groupby(['fuel', 'Grade'])['차대번호'].count()
num_by_grade = num_by_grade.rename('대수')
num_by_grade

fuel    Grade
경유      A        206113
        B        100004
        C        809308
        D         37272
기타      X            43
휘발유_가스  A         43624
        B        297398
        X         17449
Name: 대수, dtype: int64

## [출력] EG 통계

In [200]:
today_date = datetime.today().strftime("%Y.%m.%d")
today_date

'2023.02.21'

In [201]:
with pd.ExcelWriter(f'analysis2/Case5/[BD1][G{grade}][통계]Case5_등급분류(중복차대제거)(말소제거)({today_date}).xlsx') as writer:
    num_by_fuel.to_excel(writer, sheet_name='연료별', index=False)
    num_by_grade.to_excel(writer, sheet_name='등급별')

In [211]:
df.columns

Index(['차량관리번호', '차량등록번호', '본거지법정동코드', '소유자구분', '차종_등록정보', '용도', '현재회원번호',
       '차대번호', '최초등록일자', '차량연식', '차량제작일자', '배출가스검사유효일자', '제원관리번호', '배출가스인증번호',
       '배출가스등급', '저공해조치YN(미사용)', '저공해조치종류', '저공해조치구조변경YN', '차량등록번호숫자정보',
       '차량말소YN', '최초회원번호', '취득일자', '취득금액', '생성일시_등록정보', '수정일시_등록정보', '차명',
       '차종_제원정보', '차종분류', '차종유형(소분류)', '자동차형식', '제작사명', '연료', '엔진형식', '공차중량',
       '총중량', '적재중량', '엔진출력', '배기량', '승차인원', '차량길이', '차량너비', '차량높이', '원산지명',
       '삭제YN', '승인일자', '구동형식', '변속기종류', '생성일시_제원정보', '수정일시_제원정보', '저감장치종류',
       'TDMS부착여부', '구조변경일자', '부착업체명', '부착일자', '탈거승인일', '차량번호', '저감장치반납여부',
       '저감장치구분', '삼원촉매장치반납여부', 'DPF유무', '법정동코드', '시도', '시구군', '읍면동', '리',
       '지역3', '폐지여부', '대기관리권역', '대기관리권역YN'],
      dtype='object')

## [출력] 지역별 샘플

In [239]:
df[['행정구역코드', '시도', '시구군', 'Grade', 'DPF유무', '차량연식']].tail(10)

,행정구역코드,시도,시구군,Grade,DPF유무,차량연식
1511201,44200,충남,아산시,B,무,1987
1511202,28710,인천,강화군,B,무,1987
1511203,44150,충남,공주시,B,무,1987
1511204,11560,서울,영등포구,B,무,1987
1511205,11680,서울,강남구,X,무,2007
1511206,41550,경기,안성시,B,무,1995
1511207,50110,제주,제주시,B,무,1987
1511208,11170,서울,용산구,A,무,2000
1511209,11650,서울,서초구,B,무,1968
1511210,11590,서울,동작구,X,무,2002


In [240]:
df[['행정구역코드', '시도', '시구군', 'Grade', 'DPF유무', '차량연식']].tail(10).to_excel(f'analysis2/Case5/샘플_법정동코드_지역({today_date}).xlsx', index=False)